In [66]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import math
import copy

dataset = pd.read_csv('tennis.csv')
X = dataset.iloc[:, 1:].values
print(X, end="\n")
attribute = ['outlook', 'temp', 'humidity', 'wind']
print(attribute)

[['Sunny' 'Hot' 'High' 'Weak' 'No']
 ['Sunny' 'Hot' 'High' 'Strong' 'No']
 ['Overcast' 'Hot' 'High' 'Weak' 'Yes']
 ['Rain' 'Mild' 'High' 'Weak' 'Yes']
 ['Rain' 'Cool' 'Normal' 'Weak' 'Yes']
 ['Rain' 'Cool' 'Normal' 'Strong' 'No']
 ['Overcast' 'Cool' 'Normal' 'Strong' 'Yes']
 ['Sunny' 'Mild' 'High' 'Weak' 'No']
 ['Sunny' 'Cool' 'Normal' 'Weak' 'Yes']
 ['Rain' 'Mild' 'Normal' 'Weak' 'Yes']
 ['Sunny' 'Mild' 'Normal' 'Strong' 'Yes']
 ['Overcast' 'Mild' 'High' 'Strong' 'Yes']
 ['Overcast' 'Hot' 'Normal' 'Weak' 'Yes']
 ['Rain' 'Mild' 'High' 'Strong' 'No']]
['outlook', 'temp', 'humidity', 'wind']


In [67]:
class Node(object):
  def __init__(self):
    self.value = None 
    self.decision = None 
    self.childs = None


In [68]:
def findEntropy(data,rows):
  yes = 0
  no = 0
  ans = -1
  idx = len(data[0])-1
  entropy = 0
  for i in rows:
    if data[i][idx] == 'Yes':
      yes = yes + 1
    else:
      no = no + 1
  
  x = yes / (yes + no)
  y = no / (yes + no)
  if x != 0 and y != 0:
    entropy = -1 * (x * math.log2(x) + y * math.log2(y))
  if x == 1:
    ans = 1
  if y == 1:
    ans = 0
  return entropy, ans

rows = [i for i in range(0,14)]
print(rows)
entropy,ans = findEntropy(X,rows)
print(entropy)
print(ans,"\n")


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
0.9402859586706309
-1 



In [69]:
def findMaxGain(data, rows, columns):
  maxGain = 0
  retidx = -1
  entropy, ans = findEntropy(data, rows)
  if entropy == 0:
    return maxGain,retidx,ans

  #ví dụ thuộc tính outlook có 5 thuộc tính con sunny, 4 overcast và 5 rain
  for j in columns:
    mydict = {}
    idx = j
    for i in rows:
      key = data[i][idx]
      if key not in mydict:
        mydict[key] = 1
      else:
        mydict[key] = mydict[key] + 1
    gain = entropy

    print(mydict)

    #tính GainInfo của từng thuộc tính
    for key in mydict:
      yes = 0
      no= 0
      for k in rows:
        if data[k][j] == key :
          if data[k][-1] == 'Yes':
            yes = yes + 1
          else:
            no = no + 1
      print(yes, no)
      x = yes / (yes + no)
      y = no / (yes + no)
      print(x,y)
      if x != 0 and y != 0:
        gain += (mydict[key] * (x * math.log2(x) + y*math.log2(y)))/14
    print(gain)
    #tìm cột có chỉ số Gain Information cao nhất
    if gain > maxGain:
      maxGain = gain
      retidx = j
  return maxGain, retidx, ans
columns = [i for i in range(0,4)]
rows = [i for i in range(0,14)]
maxGain, retidx, ans = findMaxGain(X, rows, columns)


{'Sunny': 5, 'Overcast': 4, 'Rain': 5}
2 3
0.4 0.6
4 0
1.0 0.0
3 2
0.6 0.4
0.24674981977443905
{'Hot': 4, 'Mild': 6, 'Cool': 4}
2 2
0.5 0.5
4 2
0.6666666666666666 0.3333333333333333
3 1
0.75 0.25
0.02922256565895459
{'High': 7, 'Normal': 7}
3 4
0.42857142857142855 0.5714285714285714
6 1
0.8571428571428571 0.14285714285714285
0.15183550136234142
{'Weak': 8, 'Strong': 6}
6 2
0.75 0.25
3 3
0.5 0.5
0.04812703040826932


In [70]:
def buildTree(data, rows, columns):
  maxGain,idx, ans = findMaxGain(X,rows,columns)
  root = Node()
  root.childs = []
  print(maxGain)

  if maxGain == 0: 
    if ans == 1:
      root.value = 'Yes'
    else:
      root.value = 'No'
    return root

  root.value = attribute[idx]
  mydict= {}
  for i in rows:
    key = data[i][idx]
    if key not in mydict:
      mydict[key] = 1 
    else: 
      mydict[key] += 1
  
  newcolumns = copy.deepcopy(columns)
  newcolumns.remove(idx)

  for key in mydict:
    newrows = []
    for i in rows:
      if data[i][idx] == key:
        newrows.append(i)
    #print(newrows)
    temp = buildTree(data, newrows, newcolumns)
    temp.decision = key
    root.childs.append(temp)

  return root

root = buildTree(X,rows,columns)

{'Sunny': 5, 'Overcast': 4, 'Rain': 5}
2 3
0.4 0.6
4 0
1.0 0.0
3 2
0.6 0.4
0.24674981977443905
{'Hot': 4, 'Mild': 6, 'Cool': 4}
2 2
0.5 0.5
4 2
0.6666666666666666 0.3333333333333333
3 1
0.75 0.25
0.02922256565895459
{'High': 7, 'Normal': 7}
3 4
0.42857142857142855 0.5714285714285714
6 1
0.8571428571428571 0.14285714285714285
0.15183550136234142
{'Weak': 8, 'Strong': 6}
6 2
0.75 0.25
3 3
0.5 0.5
0.04812703040826932
0.24674981977443905
{'Hot': 2, 'Mild': 2, 'Cool': 1}
0 2
0.0 1.0
1 1
0.5 0.5
1 0
1.0 0.0
0.8280934515975258
{'High': 3, 'Normal': 2}
0 3
0.0 1.0
2 0
1.0 0.0
0.9709505944546686
{'Weak': 3, 'Strong': 2}
1 2
0.3333333333333333 0.6666666666666666
1 1
0.5 0.5
0.6313157728715637
0.9709505944546686
0
0
0
{'Mild': 3, 'Cool': 2}
2 1
0.6666666666666666 0.3333333333333333
1 1
0.5 0.5
0.6313157728715637
{'High': 2, 'Normal': 3}
1 1
0.5 0.5
2 1
0.6666666666666666 0.3333333333333333
0.6313157728715637
{'Weak': 3, 'Strong': 2}
3 0
1.0 0.0
0 2
0.0 1.0
0.9709505944546686
0.9709505944546686
0


In [71]:
def traverse(root):
  print(root.decision)
  print(root.value)
  n = len(root.childs)
  if n > 0:
    for i in range(0,n):
      traverse(root.childs[i])
      
root.decision = 'Start'
traverse(root)

Start
outlook
Sunny
humidity
High
No
Normal
Yes
Overcast
Yes
Rain
wind
Weak
Yes
Strong
No
